# Model/OOP Hacks

- toc: true
- badges: true
- comments: true
- categories: [jupyter]

In [7]:
from werkzeug.security import generate_password_hash, check_password_hash
import json
from datetime import date


def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))


class User:
    def __init__(self, name, classOf, dob, uid, password):
        self._name = name
        self._uid = uid
        self._dob = dob
        self._classOf = classOf
        self.set_password(password)

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, name):
        self._name = name

    @property
    def uid(self):
        return self._uid

    @uid.setter
    def uid(self, uid):
        self._uid = uid

    @property
    def dob(self):
        return self._dob

    @dob.setter
    def dob(self, dob):
        self._dob = dob

    @property
    def classOf(self):
        return self._classOf

    @classOf.setter
    def classOf(self, classOf):
        self._classOf = classOf

    def is_uid(self, uid):
        return self._uid == uid

    @property
    def password(self):
        return self._password[0:8] + "..."

    def set_password(self, password):
        self._password = generate_password_hash(password, method="sha256")

    def is_password(self, password):
        result = check_password_hash(self._password, password)
        return result

    def toJSON(self):
        excluded_fields = ["_password", "_dob"]
        return json.dumps(
            {
                k[1:]: v
                for k, v in self.__dict__.items()
                if k not in excluded_fields
            }
            | {"age": calculate_age(self._dob)},
            cls=DateTimeEncoder,
        )

    def __str__(self):
        return f'name: "{self.name}", class of: {self.classOf}, id: "{self.uid}", psw: "{self.password}"'

    def __repr__(self):
        return f"Person(name={self._name}, classOf={self._classOf}, uid={self._uid}, password={self._password})"


def tester(users, uid, psw):
    result = None
    for user in users:
        if user.is_uid(uid) and user.is_password(psw):
            print("* ", end="")
            result = user
        print(str(user))
    return result


class DateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, date):
            return obj.isoformat()


if __name__ == "__main__":
    u1 = User(
        name="Thomas Edison",
        classOf=2024,
        dob=date(2022, 12, 25),
        uid="toby",
        password="123toby",
    )
    u2 = User(
        name="Nicholas Tesla",
        classOf=2025,
        dob=date(2021, 1, 7),
        uid="nick",
        password="123nick",
    )
    u3 = User(
        name="Alexander Graham Bell",
        classOf=2024,
        dob=date(2020, 10, 18),
        uid="lex",
        password="123lex",
    )
    u4 = User(
        name="Eli Whitney",
        classOf=2026,
        dob=date(2019, 9, 16),
        uid="eli",
        password="123eli",
    )
    u5 = User(
        name="Hedy Lemarr",
        classOf=2027,
        dob=date(2018, 3, 12),
        uid="hedy",
        password="123hedy",
    )

    users = [u1, u2, u3, u4, u5]

    print("Test 1: find user 3")
    u = tester(users, u3.uid, "123lex")

    print("\nTest 2: change user 3")
    u.name = "John Mortensen"
    u.uid = "jm1021"
    u.set_password("123qwerty")
    u = tester(users, u.uid, "123qwerty")

    print("\nTest 3: generate JSON")
    json_strings = [u.toJSON() for u in users]
    print("\n".join(json_strings))

Test 1: find user 3
name: "Thomas Edison", class of: 2024, id: "toby", psw: "sha256$9..."
name: "Nicholas Tesla", class of: 2025, id: "nick", psw: "sha256$Y..."
* name: "Alexander Graham Bell", class of: 2024, id: "lex", psw: "sha256$4..."
name: "Eli Whitney", class of: 2026, id: "eli", psw: "sha256$2..."
name: "Hedy Lemarr", class of: 2027, id: "hedy", psw: "sha256$y..."

Test 2: change user 3
name: "Thomas Edison", class of: 2024, id: "toby", psw: "sha256$9..."
name: "Nicholas Tesla", class of: 2025, id: "nick", psw: "sha256$Y..."
* name: "John Mortensen", class of: 2024, id: "jm1021", psw: "sha256$N..."
name: "Eli Whitney", class of: 2026, id: "eli", psw: "sha256$2..."
name: "Hedy Lemarr", class of: 2027, id: "hedy", psw: "sha256$y..."

Test 3: generate JSON
{"name": "Thomas Edison", "uid": "toby", "classOf": 2024, "age": 0}
{"name": "Nicholas Tesla", "uid": "nick", "classOf": 2025, "age": 2}
{"name": "John Mortensen", "uid": "jm1021", "classOf": 2024, "age": 2}
{"name": "Eli Whitne

In [8]:
import uuid
import json

class Todo:
    def __init__(self, text):
        self._text = text
        self._uuid = uuid.uuid4()
        self._completed = False

    @property
    def text(self):
        return self._text

    @text.setter
    def text(self, value):
        self._text = value

    @property
    def uuid(self):
        return self._uuid

    @property
    def completed(self):
        return self._completed

    @completed.setter
    def completed(self, value):
        self._completed = value

class TodoController:
    def __init__(self):
        self._todos = []
    
    @property
    def todos(self):
        return self._todos

    def to_json(self):
        todos_list = []
        for todo in self._todos:
            todos_list.append({
                'text': todo.text,
                'uuid': str(todo.uuid),
                'completed': todo.completed
            })
        return json.dumps(todos_list)

    def add_todo(self, text):
        todo = Todo(text)
        self._todos.append(todo)
        return todo.uuid

    def remove_todo(self, todo_id):
        self._todos = [todo for todo in self._todos if todo.uuid != todo_id]

    def toggle_todo(self, todo_id):
        for todo in self._todos:
            if todo.uuid == todo_id:
                todo.completed = not todo.completed
                break

controller = TodoController()
id = controller.add_todo("Brush my teeth")
controller.toggle_todo(id)
controller.to_json()

'[{"text": "Brush my teeth", "uuid": "a839a1ae-f44a-49fd-b935-a72d13b005ca", "completed": true}]'